# HydroPumpTurbine with Energy Model

> **Set up**
>
> To run this notebook, first install the Julia kernel for Jupyter Notebooks using [IJulia](https://julialang.github.io/IJulia.jl/stable/manual/installation/), then [create an environment](https://pkgdocs.julialang.org/v1/environments/) for this tutorial with the packages listed with `using <PackageName>` further down.
>
> This tutorial has demonstrated compatibility with these package versions. If you run into any errors, first check your package versions for consistency using `Pkg.status()`.
>
 > ```
 > Status `~/work/HydroPowerSimulations.jl/HydroPowerSimulations.jl/docs/Project.toml`
 >   [a93c6f00] DataFrames v1.8.1
 >   [864edb3b] DataStructures v0.19.3
 >   [e30172f5] Documenter v1.16.1
 >   [d12716ef] DocumenterInterLinks v1.1.0
 >   [35a29f4d] DocumenterTools v0.1.21
 >   [87dc4568] HiGHS v1.21.0
 >   [fc1677e0] HydroPowerSimulations v0.13.1 `~/work/HydroPowerSimulations.jl/HydroPowerSimulations.jl`
 >   [2cd47ed4] InfrastructureSystems v3.3.0
 >   [b6b21f68] Ipopt v1.14.0
 >   [23fbe1c1] Latexify v0.16.10
 >   [98b081ad] Literate v2.21.0
 > ⌅ [94fada2c] PowerFlows v0.13.1
 >   [e690365d] PowerSimulations v0.32.4
 >   [f00506e0] PowerSystemCaseBuilder v2.2.0
 >   [bcd98974] PowerSystems v5.4.0
 >   [08abe8d2] PrettyTables v3.1.2
 >   [9e3dc215] TimeSeries v0.25.2
 > Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated`
 > 
 > ```


> *Note*
>
>
> `HydroPowerSimulations.jl` is an extension library of [`PowerSimulations.jl`](https://nrel-sienna.github.io/PowerSimulations.jl/latest/) for modeling hydro units. Users are encouraged to review the [single-step tutorial in `PowerSimulations.jl`](https://nrel-sienna.github.io/PowerSimulations.jl/latest/tutorials/decision_problem/) before this tutorial.
>
## Load packages

In [ ]:
using PowerSystems
using PowerSimulations
using HydroPowerSimulations
using PowerSystemCaseBuilder
using HiGHS ## solver

## Data

> *Note*
>
>
> `PowerSystemCaseBuilder.jl` is a helper library that makes it easier to reproduce examples in the documentation and tutorials. Normally you would pass your local files to create the system data instead of calling the function `build_system`.
> For more details visit [PowerSystemCaseBuilder README](https://github.com/NREL-Sienna/PowerSystemCaseBuilder.jl/blob/main/README.md)

In [ ]:
sys = build_system(PSITestSystems, "c_sys5_hydro_pump_energy")

With a single `PowerSystems.HydroPumpTurbine` connected to two `PowerSystems.HydroReservoir` (head and tail reservoirs of the turbine):

In [ ]:
hy = only(get_components(HydroPumpTurbine, sys))

res_head = get_component(HydroReservoir, sys, "Bat_head_reservoir")
res_tail = get_component(HydroReservoir, sys, "Bat_tail_reservoir")

Note that the reservoirs has a `level_data_type` of `ENERGY`, that implies its storage level limits data are in MWh. That means that the available capacity of the head reservoir is between 5.0 and 400 MWh, while the tail reservoir is set to zero, implying an infinite tail reservoir.

## Decision Model

Setting up the formulations based on [`PowerSimulations.jl`](https://nrel-sienna.github.io/PowerSimulations.jl/latest/formulation_library/Introduction/):

In [ ]:
template = ProblemTemplate(PTDFPowerModel)
set_device_model!(template, ThermalStandard, ThermalBasicDispatch)
set_device_model!(template, PowerLoad, StaticPowerLoad)
set_device_model!(template, Line, StaticBranch)

but, now we also include the HydroTurbine using `HydroPumpEnergyDispatch`:

In [ ]:
pump_model = DeviceModel(
    HydroPumpTurbine,
    HydroPumpEnergyDispatch;
    attributes = Dict{String, Any}(
        "reservation" => true,
        "energy_target" => false,
    ),
    time_series_names = Dict(),
)
set_device_model!(template, pump_model)

The `HydroPumpEnergyDispatch`(@ref) is a closed model for turbine and must be connected to independent reservoirs (not connected with other `HydroTurbine`). For that reason it is not needed to include a model of `HydroEnergyModelReservoir`. When the attribute `reservation` is set-up to `true` it does not allow to simultaneously use the pump and turbine, forcing one of those variables to zero. The `energy_target` attributes allow to include a final target for the head reservoir based on its `level_targets` field.

In addition, the `time_series_names` is set-up to an empty dictionary. By default, the `HydroPumpEnergyDispatch`(@ref) model allows to include limits on the `capacity` and `max_active_power` at each time step if the user need it by properly setting up those time series (similar to a `HydroDispatchRunOfRiver` model)

```julia
time_series_names = Dict(
    ActivePowerTimeSeriesParameter => "max_active_power",
    EnergyCapacityTimeSeriesParameter => "capacity",
)
```

With the template properly set-up, we construct, build and solve the optimization problem:

In [ ]:
model = DecisionModel(template, sys; optimizer = HiGHS.Optimizer)
build!(model; output_dir = mktempdir())
solve!(model)

## Exploring Results

Results can be explored using:

In [ ]:
res = OptimizationProblemResults(model)

Use `read_variable` to read in the dispatch variable results for the hydro:

In [ ]:
var = read_variable(
    res,
    "ActivePowerVariable__HydroPumpTurbine";
    table_format = TableFormat.WIDE,
)

its pump usage

In [ ]:
var = read_variable(
    res,
    "ActivePowerPumpVariable__HydroPumpTurbine";
    table_format = TableFormat.WIDE,
)

and stored energy:

In [ ]:
var =
    read_variable(res, "EnergyVariable__HydroPumpTurbine"; table_format = TableFormat.WIDE)